In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K


In [2]:
base_dir = 'data'

# F1 score metric


In [3]:
def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return K.mean(f1)

# Function to prepare the directory structure


In [4]:
def prepare_data_directory(base_dir):
    temp_dir = 'temp_data'
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    for plant in os.listdir(base_dir):
        plant_path = os.path.join(base_dir, plant)
        if os.path.isdir(plant_path):
            for class_type in ['diseased', 'healthy']:
                class_path = os.path.join(plant_path, class_type)
                if os.path.isdir(class_path):
                    target_dir = os.path.join(temp_dir, class_type)
                    os.makedirs(target_dir, exist_ok=True)
                    for img in os.listdir(class_path):
                        img_path = os.path.join(class_path, img)
                        if os.path.isfile(img_path):
                            shutil.copy(img_path, os.path.join(target_dir, f"{plant}_{img}"))
    return temp_dir

# Prepare the data directory


In [ ]:
temp_dir = prepare_data_directory(base_dir)


# Create ImageDataGenerators


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split the data into training and validation sets
)

# Create training and validation generators


In [ ]:
train_generator = train_datagen.flow_from_directory(
    temp_dir,
    target_size=(150, 150),
    batch_size=4,
    class_mode='binary',
    subset='training'
)

In [ ]:
validation_generator = train_datagen.flow_from_directory(
    temp_dir,
    target_size=(150, 150),
    batch_size=4,
    class_mode='binary',
    subset='validation'
)

# Debugging: Print number of samples


In [10]:
print(f'Number of training samples: {train_generator.samples}')
print(f'Number of validation samples: {validation_generator.samples}')
print(f'Classes: {train_generator.class_indices}')


Number of training samples: 3602
Number of validation samples: 900
Classes: {'diseased': 0, 'healthy': 1}


# Build the model


In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

C:\Users\mmm-n\anaconda3\envs\ai\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compile the model


In [12]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy', f1_score])

# Define callbacks


In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


# Train the model


In [14]:
validation_generator.samples // validation_generator.batch_size

28

In [15]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=1,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/3


C:\Users\mmm-n\anaconda3\envs\ai\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


112/112 ━━━━━━━━━━━━━━━━━━━━ 654s 5s/step - accuracy: 0.5610 - f1_score: 0.2769 - loss: 0.6765 - val_accuracy: 0.7500 - val_f1_score: 0.3431 - val_loss: 0.6624
Epoch 2/3
  1/112 ━━━━━━━━━━━━━━━━━━━━ 14s 127ms/step - accuracy: 0.6875 - f1_score: 0.0972 - loss: 0.5381

C:\Users\mmm-n\anaconda3\envs\ai\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


112/112 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.6875 - f1_score: 0.0490 - loss: 0.5381 - val_accuracy: 0.5625 - val_f1_score: 0.2976 - val_loss: 0.7148
Epoch 3/3
112/112 ━━━━━━━━━━━━━━━━━━━━ 628s 5s/step - accuracy: 0.6855 - f1_score: 0.3402 - loss: 0.6095 - val_accuracy: 0.5000 - val_f1_score: 0.4613 - val_loss: 0.6896


# Load the best model


In [18]:
model.load_weights('best_model.keras')


# Evaluate the model


In [19]:
loss, accuracy, f1 = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy*100:.2f}%')
print(f'Validation F1 Score: {f1*100:.2f}%')


29/29 ━━━━━━━━━━━━━━━━━━━━ 158s 5s/step - accuracy: 0.6815 - f1_score: 0.2375 - loss: 0.6424
Validation Accuracy: 68.22%
Validation F1 Score: 23.75%


# predictions

In [25]:
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    return 'Diseased' if prediction < 0.5 else 'Healthy'

In [27]:
# image_path = './data/HAH/Healthy/15.png'
# result = predict_image(image_path)
# print(f'The plant is {result}')


In [ ]:
import os
import hashlib
from flask import Flask, request, jsonify
from PIL import Image
import io

app = Flask(__name__)

In [ ]:
store_folder = 'store'
os.makedirs(store_folder, exist_ok=True)

def save_image(image):
    image_hash = hashlib.sha256(image).hexdigest()
    
    image_path = os.path.join(store_folder, f"{image_hash}.jpg")
    with open(image_path, 'wb') as f:
        f.write(image)
    
    return image_path

In [28]:
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file provided'}), 400

    image_file = request.files['image']
    try:
        image = Image.open(image_file)
        image_byte_array = io.BytesIO()
        image.save(image_byte_array, format=image.format)
        image_bytes = image_byte_array.getvalue()
        image_path = save_image(image_bytes)
        result = predict_image(image_path)
        
        return jsonify({'result': result})
    except Exception as e:
        return jsonify({'error': 'Invalid image file', 'message': str(e)}), 400

In [ ]:
app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


127.0.0.1 - - [21/May/2024 17:23:58] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


127.0.0.1 - - [21/May/2024 17:34:48] "POST /predict HTTP/1.1" 200 -


In [6]:
expr1 = MathTex("a + a + b + b^2 - a^2")

In [1]:
from manim import *

class CombineLikeTerms(Scene):
    def construct(self):
        # Title at the top
        title = Text("How to Solve Like Term Problems in Math")
        title.to_edge(UP)
        
        # First example
        expr1 = MathTex("a + a + b + b^2 - a^2")
        simplified_expr1 = MathTex("2a + b + b^2 - a^2")
        
        # Position the first example expressions
        expr1.next_to(title, DOWN, buff=1)
        simplified_expr1.next_to(expr1, DOWN, buff=1)

        # Display title and first original expression
        self.play(Write(title))
        self.wait(1)
        self.play(Write(expr1))
        self.wait(2)
        
        # Highlight like terms (a + a)
        framebox1 = SurroundingRectangle(expr1[0][0:3], buff=.1)
        self.play(Create(framebox1))
        self.wait(1)
        
        # Transform to the first simplified expression
        self.play(ReplacementTransform(expr1, simplified_expr1), FadeOut(framebox1))
        self.wait(2)
        
        # Highlight the simplified part 2a
        framebox2 = SurroundingRectangle(simplified_expr1[0][0:2], buff=.1)
        self.play(Create(framebox2))
        self.wait(2)

        # Extra example
        expr2 = MathTex("3x + 2y - x + 5y")
        simplified_expr2 = MathTex("2x + 7y")
        
        # Position the second example expressions
        expr2.next_to(simplified_expr1, DOWN, buff=2)
        simplified_expr2.next_to(expr2, DOWN, buff=1)

        # Display the second original expression
        self.play(Write(expr2))
        self.wait(2)
        
        # Highlight like terms (3x - x and 2y + 5y)
        framebox3 = SurroundingRectangle(expr2[0][0:3], buff=.1)
        framebox4 = SurroundingRectangle(expr2[0][9:], buff=.1)
        self.play(Create(framebox3))
        self.wait(1)
        self.play(Create(framebox4))
        self.wait(1)
        
        # Transform to the second simplified expression
        self.play(ReplacementTransform(expr2, simplified_expr2), FadeOut(framebox3), FadeOut(framebox4))
        self.wait(2)
        
        # Highlight the simplified parts 2x and 7y
        framebox5 = SurroundingRectangle(simplified_expr2[0][0:2], buff=.1)
        framebox6 = SurroundingRectangle(simplified_expr2[0][4:], buff=.1)
        self.play(Create(framebox5))
        self.wait(1)
        self.play(Create(framebox6))
        self.wait(2)


In [3]:
a=CombineLikeTerms()

In [ ]:
pwd

In [5]:
a.construct()

[06/18/24 17:29:04] INFO     Writing a + a + b + b^2 - a^2 to                               ]8;id=885205;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=143381;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\utils\tex_file_writing.py#109\109]8;;\
                             media\Tex\9477b208d05fae0e.tex                                                        

[06/18/24 17:29:05] INFO     Writing 2a + b + b^2 - a^2 to media\Tex\d7b280c68ee467cc.tex   ]8;id=955014;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=670563;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\utils\tex_file_writing.py#109\109]8;;\

[06/18/24 17:29:08] INFO     Animation 0 : Partial movie file written in                   ]8;id=347211;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=238202;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\3977891868_2002733993_22313                         
                             2457.mp4'                                                                             

[06/18/24 17:29:09] INFO     Animation 1 : Partial movie file written in                   ]8;id=823625;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=257586;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\2852726489_2268332985_11829                         
                             27419.mp4'                                                                            

[06/18/24 17:29:10] INFO     Animation 2 : Partial movie file written in                   ]8;id=698600;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=327034;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\2852726489_125122034_118831                         
                             8917.mp4'                                                                             

...

[06/18/24 17:29:21] INFO     Animation 12 : Partial movie file written in                  ]8;id=87973;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=28931;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\2852726489_1449414708_41227                         
                             38062.mp4'                                                                            

                    INFO     Animation 13 : Partial movie file written in                  ]8;id=457458;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=635349;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\2852726489_2268332985_28441                         
                             99684.mp4'                                                                            

[06/18/24 17:29:22] INFO     Animation 14 : Partial movie file written in                  ]8;id=530088;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=967658;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\2852726489_2160398502_22957                         
                             58179.mp4'                                                                            

[06/18/24 17:29:23] INFO     Animation 15 : Partial movie file written in                  ]8;id=210466;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=438092;file://C:\Users\mmm-n\anaconda3\Lib\site-packages\manim\scene\scene_file_writer.py#527\527]8;;\
                             'D:\CivilWar\Project\AI\PlantCare\H5\media\videos\1080p60\par                         
                             tial_movie_files\CombineLikeTerms\2852726489_2268332985_39467                         
                             32753.mp4'                                                                            

KeyboardInterrupt: 